In [447]:
import numpy as np

import pandas as pd

import re
import srt
import time
from datetime import datetime, timedelta

1. Set the date and session directory
2. Choose the person to process

In [448]:
all_date=["12-15-2021","01-28-2022","02-11-2022","03-04-2022","03-05-2022","03-11-2022","03-12-2022"]
date=all_date[5]
total_session=['1','2','3']
all_pc=['1','2','3']

In [449]:
date

'03-11-2022'

In [450]:
for session in total_session:
    for pc in all_pc:
        sentences=[]
        data="../3people/"+date+"/subtitles/Session_"+session+"_PC_"+pc+"_Captions."
        clean_data="../3people/"+date+"/Caption/Session_"+session+"_PC_"+pc
        f = open(data+"vtt", "r",encoding="utf-8")
        cc= open(data+"srt","r",encoding="utf-8")
        subs = list(srt.parse(cc))
        for line in f:
            sentences.append(line.replace("\n", ""))
        zero = datetime(1999, 1, 1,0,0,0,000000)
        all_sentences = pd.DataFrame(columns=['start', 'end', 'Sentence'])
        all_words_by_sentence = pd.DataFrame(columns=['id', 'start', 'end', 'word'])
        empty_word = {'id':'','start':'','end':'','word':''}
        sub_index=0
        for i in range(len(sentences)):
            start=zero+subs[sub_index].start
            start=start.strftime('%H:%M:%S.%f')[:-3]
            end=zero+subs[sub_index].end
            end=end.strftime('%H:%M:%S.%f')[:-3]
            if start in sentences[i][:12]:
                ID=0
                if "<c>" in sentences[i+2]:


                    a= re.split('<c> |</c>|<|>',sentences[i+2])
                    while("" in a):
                        a.remove("")
                    sentence=None
                    element = datetime.strptime(a[len(a)-2], '%H:%M:%S.%f')
                    result_1 = element + timedelta(milliseconds=200)
                    for k in range(len(a)):
                        if k==len(a)-1:
                            sentence= sentence +" " + a[k]
                            df1={'id':str(ID),'start':a[k-1],'end':result_1.strftime('%H:%M:%S.%f')[:-3],'word':a[k]}
                            ID=ID+1
                            all_words_by_sentence = all_words_by_sentence.append(df1, ignore_index = True)
                        elif k==0:
                            sentence= a[k] 
                            df1={'id':str(ID),'start':start,'end':a[k+1],'word':a[k]}
                            ID=ID+1
                            all_words_by_sentence = all_words_by_sentence.append(df1, ignore_index = True)
                        elif k % 2 == 0:
                            sentence= sentence +" " + a[k]
                            df1={'id':str(ID),'start':a[k-1],'end':a[k+1],'word':a[k]}
                            ID=ID+1
                            all_words_by_sentence = all_words_by_sentence.append(df1, ignore_index = True)

                    df2 = {'start': start, 'end': result_1.strftime('%H:%M:%S.%f')[:-3], 'Sentence': sentence}

                    all_sentences = all_sentences.append(df2, ignore_index = True)

                    all_words_by_sentence = all_words_by_sentence.append(empty_word, ignore_index = True)

                else:
                    df1 = df1={'id':str(ID),'start':start,'end':end,'word':sentences[i+2]}
                    df2 = {'start':start,'end':end, 'Sentence': sentences[i+2]}
                    all_words_by_sentence = all_words_by_sentence.append(df1, ignore_index = True)
                    all_words_by_sentence = all_words_by_sentence.append(empty_word, ignore_index = True)
                    all_sentences = all_sentences.append(df2, ignore_index = True)

                sub_index= sub_index+1
                if sub_index == len(subs):
                    break
        f.close()  
        cc.close()
        #Define threshold of gap
        threshold = timedelta(milliseconds=200)
        for i in range(len(all_sentences)-1, 0, -1):
            if datetime.strptime(all_sentences['start'][i], '%H:%M:%S.%f') - datetime.strptime(all_sentences['end'][i-1], '%H:%M:%S.%f') < threshold:
                all_sentences['end'][i-1]=all_sentences['end'][i]
                all_sentences['Sentence'][i-1]= all_sentences['Sentence'][i-1] + " " + all_sentences['Sentence'][i]
                all_sentences = all_sentences.drop(i)
        all_sentences = all_sentences.reset_index()
        all_sentences=all_sentences.drop(columns=['index'])
        for i in range(len(all_words_by_sentence)-2, -1, -1):
            if all_words_by_sentence['word'][i]=="":
                if datetime.strptime(all_words_by_sentence['start'][i+1], '%H:%M:%S.%f') - datetime.strptime(all_words_by_sentence['end'][i-1], '%H:%M:%S.%f') < threshold:
                    all_words_by_sentence['end'][i-1] = all_words_by_sentence['start'][i+1] 
                    all_words_by_sentence = all_words_by_sentence.drop(i)
        all_words_by_sentence = all_words_by_sentence.reset_index()
        all_words_by_sentence=all_words_by_sentence.drop(columns=['index'])
        present= len(all_words_by_sentence)-1
        previous= present
        for i in range(len(all_words_by_sentence)-2, -1, -1):
            if all_words_by_sentence['word'][i]=="":
                present= i

                for k in range(1,previous-present):
                    all_words_by_sentence['id'][i+k]=k-1
                previous= i 
        all_sentences['SID']='0'
        all_words_by_sentence['SID']='0'

        for i in range(len(all_sentences)):
            all_sentences['SID'][i]=date+"_Session_"+session+"_PC_"+pc+"_"+str(i+1)

        k=1
        for i in range(len(all_words_by_sentence)):
            if all_words_by_sentence['word'][i]=="":
                k=k+1
                all_words_by_sentence['SID'][i]=''
            else:
                all_words_by_sentence['SID'][i]=date+"_Session_"+session+"_PC_"+pc+"_"+str(k)

        all_sentences.to_csv(clean_data+'_sentence.csv',index=False)   
        all_words_by_sentence.to_csv(clean_data+'_words.csv',index=False) 

In [451]:
# date="03-12-2022"
# session="3"
# pc="3"
# sentences=[]
# data="../3people/"+date+"/subtitles/Session_"+session+"_PC_"+pc+"_Captions."
# clean_data="../3people/"+date+"/Caption/Session_"+session+"_PC_"+pc
# f = open(data+"vtt", "r",encoding="utf-8")
# cc= open(data+"srt","r",encoding="utf-8")
# subs = list(srt.parse(cc))
# for line in f:
#     sentences.append(line.replace("\n", ""))

Set a zero time

In [452]:
# zero = datetime(1999, 1, 1,0,0,0,000000)

In [453]:
# all_sentences = pd.DataFrame(columns=['start', 'end', 'Sentence'])
# all_words_by_sentence = pd.DataFrame(columns=['id', 'start', 'end', 'word'])
# empty_word = {'id':'','start':'','end':'','word':''}
# sub_index=0

In [454]:
# for i in range(len(sentences)):
#     start=zero+subs[sub_index].start
#     start=start.strftime('%H:%M:%S.%f')[:-3]
#     end=zero+subs[sub_index].end
#     end=end.strftime('%H:%M:%S.%f')[:-3]
#     if start in sentences[i][:12]:
#         ID=0
#         if "<c>" in sentences[i+2]:
        
            
#             a= re.split('<c> |</c>|<|>',sentences[i+2])
#             while("" in a):
#                 a.remove("")
#             sentence=None
#             element = datetime.strptime(a[len(a)-2], '%H:%M:%S.%f')
#             result_1 = element + timedelta(milliseconds=200)
#             for k in range(len(a)):
#                 if k==len(a)-1:
#                     sentence= sentence +" " + a[k]
#                     df1={'id':str(ID),'start':a[k-1],'end':result_1.strftime('%H:%M:%S.%f')[:-3],'word':a[k]}
#                     ID=ID+1
#                     all_words_by_sentence = all_words_by_sentence.append(df1, ignore_index = True)
#                 elif k==0:
#                     sentence= a[k] 
#                     df1={'id':str(ID),'start':start,'end':a[k+1],'word':a[k]}
#                     ID=ID+1
#                     all_words_by_sentence = all_words_by_sentence.append(df1, ignore_index = True)
#                 elif k % 2 == 0:
#                     sentence= sentence +" " + a[k]
#                     df1={'id':str(ID),'start':a[k-1],'end':a[k+1],'word':a[k]}
#                     ID=ID+1
#                     all_words_by_sentence = all_words_by_sentence.append(df1, ignore_index = True)
           
#             df2 = {'start': start, 'end': result_1.strftime('%H:%M:%S.%f')[:-3], 'Sentence': sentence}

#             all_sentences = all_sentences.append(df2, ignore_index = True)
    
#             all_words_by_sentence = all_words_by_sentence.append(empty_word, ignore_index = True)
            
#         else:
#             df1 = df1={'id':str(ID),'start':start,'end':end,'word':sentences[i+2]}
#             df2 = {'start':start,'end':end, 'Sentence': sentences[i+2]}
#             all_words_by_sentence = all_words_by_sentence.append(df1, ignore_index = True)
#             all_words_by_sentence = all_words_by_sentence.append(empty_word, ignore_index = True)
#             all_sentences = all_sentences.append(df2, ignore_index = True)
            
#         sub_index= sub_index+1
#         if sub_index == len(subs):
#             break


Close open files

In [455]:
# f.close()  
# cc.close()

# Combine two sentences if the gap is too short, shorter than 200ms

In [456]:
#Define threshold of gap
# threshold = timedelta(milliseconds=200)

In [457]:
# for i in range(len(all_sentences)-1, 0, -1):
#     if datetime.strptime(all_sentences['start'][i], '%H:%M:%S.%f') - datetime.strptime(all_sentences['end'][i-1], '%H:%M:%S.%f') < threshold:
#         all_sentences['end'][i-1]=all_sentences['end'][i]
#         all_sentences['Sentence'][i-1]= all_sentences['Sentence'][i-1] + " " + all_sentences['Sentence'][i]
#         all_sentences = all_sentences.drop(i)
# all_sentences = all_sentences.reset_index()
# all_sentences=all_sentences.drop(columns=['index'])

In [458]:
# for i in range(len(all_words_by_sentence)-2, -1, -1):
#     if all_words_by_sentence['word'][i]=="":
#         if datetime.strptime(all_words_by_sentence['start'][i+1], '%H:%M:%S.%f') - datetime.strptime(all_words_by_sentence['end'][i-1], '%H:%M:%S.%f') < threshold:
#             all_words_by_sentence['end'][i-1] = all_words_by_sentence['start'][i+1] 
#             all_words_by_sentence = all_words_by_sentence.drop(i)
# all_words_by_sentence = all_words_by_sentence.reset_index()
# all_words_by_sentence=all_words_by_sentence.drop(columns=['index'])

In [459]:
# present= len(all_words_by_sentence)-1
# previous= present
# for i in range(len(all_words_by_sentence)-2, -1, -1):
#     if all_words_by_sentence['word'][i]=="":
#         present= i
        
#         for k in range(1,previous-present):
#             all_words_by_sentence['id'][i+k]=k-1
#         previous= i 

In [460]:
# all_sentences['SID']='0'
# all_words_by_sentence['SID']='0'

In [461]:

# for i in range(len(all_sentences)):
#     all_sentences['SID'][i]=date+"_Session_"+session+"_PC_"+pc+"_"+str(i+1)

In [462]:
# k=1
# for i in range(len(all_words_by_sentence)):
#     if all_words_by_sentence['word'][i]=="":
#         k=k+1
#         all_words_by_sentence['SID'][i]=''
#     else:
#         all_words_by_sentence['SID'][i]=date+"_Session_"+session+"_PC_"+pc+"_"+str(k)

Export captions

In [463]:
# all_sentences.to_csv(clean_data+'_sentence.csv',index=False)   
# all_words_by_sentence.to_csv(clean_data+'_words.csv',index=False) 